In [32]:
import mne
import scipy.io
import numpy as np
from mne.preprocessing import create_eog_epochs, compute_proj_eog, compute_current_source_density
from mne.time_frequency import tfr_multitaper
from mne import Epochs, pick_types, find_events, create_info
from mne.datasets import sample
from mne.io import read_raw_fif
import h5py

In [33]:
# Function to recursively print the structure of the .mat file
def print_structure(name, obj):
    if isinstance(obj, h5py.Dataset):
        print(f'Dataset: {name}, shape: {obj.shape}, dtype: {obj.dtype}')
    elif isinstance(obj, h5py.Group):
        print(f'Group: {name}')

In [34]:
file_path = 'ExpResult/Evaluation_20240716_114633_1_Block_01_20240716_115504.mat'
with h5py.File(file_path, 'r') as f:
    # Visit all items in the file
    f.visititems(print_structure)

Group: #refs#
Dataset: #refs#/a, shape: (2,), dtype: uint64
Dataset: Ave1_ERD, shape: (2, 160), dtype: float64
Dataset: Ave2_ERD, shape: (2, 150), dtype: float64
Dataset: Ave_data1, shape: (12, 300), dtype: float64
Dataset: Ave_data2, shape: (12, 300), dtype: float64
Dataset: Buffer_ERD, shape: (2, 160), dtype: float64
Dataset: ERD_data, shape: (12, 2, 300), dtype: float64
Group: ExpCond
Dataset: ExpCond/BlockLabel, shape: (8, 1), dtype: uint16
Dataset: ExpCond/DAQID, shape: (4, 1), dtype: uint16
Dataset: ExpCond/Duration, shape: (1, 1), dtype: float64
Dataset: ExpCond/FBband, shape: (13, 1), dtype: uint16
Dataset: ExpCond/FBch, shape: (15, 1), dtype: uint16
Dataset: ExpCond/FBkind, shape: (1, 1), dtype: float64
Dataset: ExpCond/FBname, shape: (10, 1), dtype: uint16
Dataset: ExpCond/FixedFreq, shape: (1, 1), dtype: uint8
Dataset: ExpCond/InitialFreq, shape: (1, 1), dtype: float64
Dataset: ExpCond/Num_Exp, shape: (1, 1), dtype: float64
Dataset: ExpCond/Taskkind, shape: (1, 1), dtype: fl

In [35]:
with h5py.File(file_path, 'r') as f:
    # Inspecting the structure of the .mat file
    for key in f.keys():
        print(f"{key}: {list(f[key]) if isinstance(f[key], h5py.Group) else f[key].shape}")
    #  EEG data is in 'Packet' and sampling frequency in 'ExpConfig/Fs'
    eeg_data = np.array(f['Packet'])
    sfreq = float(np.array(f['ExpConfig/Fs']))

#refs#: ['a']
Ave1_ERD: (2, 160)
Ave2_ERD: (2, 150)
Ave_data1: (12, 300)
Ave_data2: (12, 300)
Buffer_ERD: (2, 160)
ERD_data: (12, 2, 300)
ExpCond: ['BlockLabel', 'DAQID', 'Duration', 'FBband', 'FBch', 'FBkind', 'FBname', 'FixedFreq', 'InitialFreq', 'Num_Exp', 'Taskkind', 'Taskname', 'comptID', 'expstr', 'subStr']
ExpConfig: ['Fs', 'T', 'count_sample_finish', 'count_sample_rest1', 'count_sample_rest2', 'dur', 'flag_FB', 'flag_dur', 'flag_ready', 'flag_task', 'flag_trl', 'idx_Trig', 'idx_Trig2', 'num_trl', 'sign_dir_FB', 'timeTbl', 'time_blank', 'time_flag', 'time_ready', 'time_rest', 'timecount_taskonset', 'timecounter', 'timecounter_baseline', 'timecounter_finish', 'timecounter_resting']
FB_data: (12, 300)
Fs_DAQ: (1, 1)
Packet: (1264, 362100)
Ref0: (30, 2)
Ref1: (50, 2)
TF: (12, 300, 2, 51)
filtPara: ['Band', 'Band2', 'COI', 'FOI', 'bpa', 'bpb', 'nca', 'ncb', 'notch']
list_th: (1, 13)
randStream: ['Seed', 'State', 'Type']
score: (12, 2, 2)
score_FB: (12, 1)
sign_dir: (1, 1)
sign_dir_F

/var/folders/zk/lv_zrs9910s49_109vbgf6c40000gn/T/ipykernel_59810/365418674.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sfreq = float(np.array(f['ExpConfig/Fs']))


In [36]:
# Create MNE Info object and RawArray
ch_names = [f'EEG{i+1}' for i in range(eeg_data.shape[0])]
info = create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
raw = mne.io.RawArray(eeg_data, info)

Creating RawArray with float64 data, n_channels=1264, n_times=362100
    Range : 0 ... 362099 =      0.000 ...   362.099 secs
Ready.


In [38]:
ch_names

['EEG1',
 'EEG2',
 'EEG3',
 'EEG4',
 'EEG5',
 'EEG6',
 'EEG7',
 'EEG8',
 'EEG9',
 'EEG10',
 'EEG11',
 'EEG12',
 'EEG13',
 'EEG14',
 'EEG15',
 'EEG16',
 'EEG17',
 'EEG18',
 'EEG19',
 'EEG20',
 'EEG21',
 'EEG22',
 'EEG23',
 'EEG24',
 'EEG25',
 'EEG26',
 'EEG27',
 'EEG28',
 'EEG29',
 'EEG30',
 'EEG31',
 'EEG32',
 'EEG33',
 'EEG34',
 'EEG35',
 'EEG36',
 'EEG37',
 'EEG38',
 'EEG39',
 'EEG40',
 'EEG41',
 'EEG42',
 'EEG43',
 'EEG44',
 'EEG45',
 'EEG46',
 'EEG47',
 'EEG48',
 'EEG49',
 'EEG50',
 'EEG51',
 'EEG52',
 'EEG53',
 'EEG54',
 'EEG55',
 'EEG56',
 'EEG57',
 'EEG58',
 'EEG59',
 'EEG60',
 'EEG61',
 'EEG62',
 'EEG63',
 'EEG64',
 'EEG65',
 'EEG66',
 'EEG67',
 'EEG68',
 'EEG69',
 'EEG70',
 'EEG71',
 'EEG72',
 'EEG73',
 'EEG74',
 'EEG75',
 'EEG76',
 'EEG77',
 'EEG78',
 'EEG79',
 'EEG80',
 'EEG81',
 'EEG82',
 'EEG83',
 'EEG84',
 'EEG85',
 'EEG86',
 'EEG87',
 'EEG88',
 'EEG89',
 'EEG90',
 'EEG91',
 'EEG92',
 'EEG93',
 'EEG94',
 'EEG95',
 'EEG96',
 'EEG97',
 'EEG98',
 'EEG99',
 'EEG100',
 'EEG101

In [37]:
# Apply Hjorth-style Laplacian spatial filter (assuming data is from SM1 area)
# Rename channels to match standard montage
montage = mne.channels.make_standard_montage('standard_1020')
standard_ch_names = montage.ch_names[:len(ch_names)] 
rename_dict = {ch_name: standard_ch_name for ch_name, standard_ch_name in zip(ch_names, standard_ch_names)}
raw.rename_channels(rename_dict)

raw.set_montage(montage)

ValueError: DigMontage is only a subset of info. There are 1170 channel positions not present in the DigMontage. The channels missing from the montage are:

['EEG95', 'EEG96', 'EEG97', 'EEG98', 'EEG99', 'EEG100', 'EEG101', 'EEG102', 'EEG103', 'EEG104', 'EEG105', 'EEG106', 'EEG107', 'EEG108', 'EEG109', 'EEG110', 'EEG111', 'EEG112', 'EEG113', 'EEG114', 'EEG115', 'EEG116', 'EEG117', 'EEG118', 'EEG119', 'EEG120', 'EEG121', 'EEG122', 'EEG123', 'EEG124', 'EEG125', 'EEG126', 'EEG127', 'EEG128', 'EEG129', 'EEG130', 'EEG131', 'EEG132', 'EEG133', 'EEG134', 'EEG135', 'EEG136', 'EEG137', 'EEG138', 'EEG139', 'EEG140', 'EEG141', 'EEG142', 'EEG143', 'EEG144', 'EEG145', 'EEG146', 'EEG147', 'EEG148', 'EEG149', 'EEG150', 'EEG151', 'EEG152', 'EEG153', 'EEG154', 'EEG155', 'EEG156', 'EEG157', 'EEG158', 'EEG159', 'EEG160', 'EEG161', 'EEG162', 'EEG163', 'EEG164', 'EEG165', 'EEG166', 'EEG167', 'EEG168', 'EEG169', 'EEG170', 'EEG171', 'EEG172', 'EEG173', 'EEG174', 'EEG175', 'EEG176', 'EEG177', 'EEG178', 'EEG179

In [ ]:
# Apply bandpass and notch filters
raw.filter(1., 40., fir_design='firwin')
raw.notch_filter(np.arange(50, 251, 50), fir_design='firwin')

In [ ]:
laplacian = compute_current_source_density(raw)